In [ ]:
import sys
import os
import xarray as xr
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime, timedelta
import cartopy.crs as ccrs
import cartopy.feature as cfeat
from cartopy.util import add_cyclic_point
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import warnings
import psutil
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
from matplotlib import patches
from matplotlib import cm

from metpy import calc as mpcalc
from metpy.units import units

In [ ]:
Gridsat_dir = '/data/willytsai/gridsat_BT/remap_0.1deg/'

In [ ]:
os.chdir(Gridsat_dir)
data = xr.open_dataset('LandSeaMask_0.1deg.nc4')
lsmask = data.sel(lat=slice(-10,10)).landseamask
lsmask_1d = lsmask.values.flatten()

In [ ]:
plt.contourf(lsmask.T)
plt.colorbar()

In [ ]:
os.chdir(Gridsat_dir+'2014_2018/')

In [ ]:
files = sorted(glob('*nc'))

In [ ]:
%%time
tbb_bins = np.linspace(180,300,61)
tbb_dist = np.zeros(len(tbb_bins)-1)

for file in files:
    data = xr.open_dataset(file)
    tbb = data.sel(lat=slice(-10,10)).irwin_cdr
    tbb_1d = tbb.values.flatten()
    tbb_1d[lsmask_1d<100] = np.nan
    
    for i in range(len(tbb_bins)-1):
        idx = np.where(np.logical_and(tbb_1d>=tbb_bins[i],tbb_1d<tbb_bins[i+1]))[0]
        tbb_dist[i] += len(idx)

In [ ]:
from matplotlib.ticker import NullFormatter

In [ ]:
fig = plt.figure(figsize=(5,5))
plt.step(tbb_bins[:-1],np.cumsum(tbb_dist)/np.sum(tbb_dist))
plt.yscale('logit')
plt.gca().yaxis.set_minor_formatter(NullFormatter())
plt.ylim([0,0.99])
plt.ylabel('Cumulative probability',fontsize=12);
plt.xlabel('Brightness temperature [K]',fontsize=12)
plt.xlim([175,305])
#plt.yticks([0.01,0.1,1,5,10,20,30,40,50,70,80.90,95,99,99.9,99.99])
plt.grid()
plt.show

In [ ]:
os.chdir('/data/willytsai/ISCCP/fig/')
fig.savefig('TBB_cumulative_dist.pdf',bbox_inches='tight')